In [20]:
import pandas as pd
import numpy as np

import datasets

In [29]:
books = pd.read_csv("BX-Books.csv", sep=";", encoding="latin1", on_bad_lines='skip')
users = pd.read_csv("BX-Users.csv", sep=";", encoding="latin1", on_bad_lines='skip')
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="latin1", on_bad_lines='skip')

C:\Users\USER\AppData\Local\Temp\ipykernel_21500\171221850.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv", sep=";", encoding="latin1", on_bad_lines='skip')


BOOKS

In [34]:
books.head(2)

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [31]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [32]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]

In [33]:
books.rename(columns={
        'Book-Title': 'title',
        'Book-Author': 'author',
        'Year-Of-Publication': 'year',
        'Publisher': 'publisher',
        'Image-URL-L': 'image_url'
}, inplace=True)


USERS

In [37]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [36]:
users.rename(columns={'User-ID': 'user_id',
                    'Location': 'location',
                    'Age': 'age'}, inplace=True)

RATINGS

In [40]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [39]:
ratings.rename(columns={'User-ID': 'user_id',
                    'Book-Rating': 'rating'}, inplace=True)

In [49]:
#users who had at least rated more than 200 books
x = ratings['user_id'].value_counts() >= 200


In [51]:
y = x[x].index


In [54]:
ratings = ratings[ratings['user_id'].isin(y)]

In [55]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [57]:
#merge the ratings with books
ratings_with_books = ratings.merge(books, on='ISBN')

In [61]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [67]:
#no. of ratings with the book title
ratings_per_book = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [70]:
ratings_per_book.head(2)

,title,number_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [69]:
ratings_per_book.rename(columns={'rating': 'number_of_rating'}, inplace=True)

In [71]:
final_rating = ratings_with_books.merge(ratings_per_book, on='title')

In [75]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133


In [74]:
final_rating = final_rating[final_rating['number_of_rating'] >= 50]

In [76]:
#drop duplicates
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)

In [77]:
#check for null
final_rating.isnull().sum()

user_id             0
ISBN                0
rating              0
title               0
author              0
year                0
publisher           0
image_url           0
number_of_rating    0
dtype: int64